# Gestructureerde data: Complexere architecturen

In de voorgaande onderdelen heb je reeds gemerkt dat het verwerken van de input in het geval van gestructureerde data kan leiden tot een complexere netwerkarchitectuur.
In deze notebook gaan we dit in meer detail bestuderen en ook kijken naar de mogelijkheden aan de output-kant van het netwerk.

## Inputkant

Voor dit deel verwijs ik door naar de vorige notebook over het preprocessen van de input. Vooral in het geval van pytorch is er niet veel mogelijkheden om dit in pytorch zelf te manipuleren.

In tensorflow is het belangrijk stil te staan bij het feit dat er twee mogelijkheden zijn om een model te maken.
* Sequential API
* Functional API

De sequential API verwacht dat alle data door alle lagen gestuurd wordt. Hierdoor is differentiatie in de input niet mogelijk. De functional API daarentegen is flexibeler en maakt deze differentiatie wel mogelijk door met verschillende input-tensors te werken die elk op hun eigen manier verwerkt kunnen worden.

Een voorbeeld van beide methodes staat hieronder

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load Titanic dataset
titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")

# Preprocess data: handle missing values and convert categorical data
titanic.fillna({'age': titanic['age'].median(), 'embark_town': titanic['embark_town'].mode()[0]}, inplace=True)
titanic['sex'] = titanic['sex'].astype('category').cat.codes
titanic['class'] = titanic['class'].astype('category').cat.codes
titanic['embark_town'] = titanic['embark_town'].astype('category').cat.codes

# Custom PyTorch Dataset
class TitanicDataset(Dataset):
    def __init__(self, dataframe):
        # Separate numerical and categorical features
        self.num_features = dataframe[['age', 'n_siblings_spouses', 'parch', 'fare']]
        self.cat_features = dataframe[['class', 'sex', 'embark_town']]
        self.target = dataframe['survived']

        # Convert to tensors
        self.num_features = torch.tensor(self.num_features.values, dtype=torch.float32)
        self.cat_features = torch.tensor(self.cat_features.values, dtype=torch.float32)
        self.target = torch.tensor(self.target.values, dtype=torch.float32)

    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        return self.num_features[idx], self.cat_features[idx], self.target[idx]

# Create the dataset
dataset = TitanicDataset(titanic)

# Split dataset into training and testing datasets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)

# Standardize numerical features
scaler = StandardScaler()
scaler.fit(dataset.num_features)

# Create PyTorch model with multiple inputs
class ComplexModel(torch.nn.Module):
    def __init__(self, num_features, num_categories):
        super(ComplexModel, self).__init__()
        # Layers for numerical features
        self.num_layers = torch.nn.Sequential(
            torch.nn.Linear(num_features, 32),
            torch.nn.ReLU()
        )
        # Layers for categorical features
        self.cat_layers = torch.nn.Sequential(
            torch.nn.Linear(num_categories, 10),
            torch.nn.ReLU()
        )
        
        # Output layers
        self.shared_layers = torch.nn.Sequential(
            torch.nn.Linear(42, 1)
        )

    def forward(self, num_input, cat_input):
        num_out = self.num_layers(num_input)
        cat_out = self.cat_layers(cat_input)
        combined = torch.cat((num_out, cat_out), dim=1)
        return self.shared_layers(combined)

# Define the model
model = ComplexModel(num_features=dataset.num_features.shape[1], num_categories=len(dataset.cat_features.unique(dim=0)[0]))

# Define loss function and optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for i, (num_data, cat_data, target) in enumerate(train_loader):
        # Standardize numerical features
        num_data = torch.tensor(scaler.transform(num_data.numpy()))

        # Forward pass
        output = model(num_data, cat_data)
        loss = criterion(output, target.unsqueeze(-1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras import Input
from keras import layers

titanic_features = titanic.copy()
titanic_labels = titanic_features.pop('survived')

inputs = {}

for name, column in titanic_features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = Input(shape=(1,), name=name, dtype=dtype)

# selecteer gelijkaardige inputs
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}
categoric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.string}

# om alle verschillende eindresultaten bij te houden
preprocessed_inputs = []

# preprocess numeriek
x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(titanic[numeric_inputs.keys()]))

preprocessed_inputs.append(norm(x))

# preprocess categoriek
for name, input in categoric_inputs.items():
  lookup = layers.StringLookup(vocabulary=np.unique(titanic_features[name]))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
    
  preprocessed_inputs.append(x)

# voeg alle verwerkte inputs samen
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

x = layers.Dense(64, activation='relu')(preprocessed_inputs_cat)
output = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, output)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(titanic_features, titanic_labels, test_size=0.2, random_state=42)
titanic_features.info()

# Convert pandas DataFrames to TensorFlow Datasets
X_train_dict = {name: np.array(value) for name, value in X_train.items()}
X_test_dict = {name: np.array(value) for name, value in X_test.items()}

# Train the model
history = model.fit(X_train_dict, y_train, epochs=10)

## Meerdere outputs/acties door 1 model

In sommige gevallen kan het nodig zijn dat je simultaan meerdere voorspellingen doet voor dezelfde data.
In plaats van twee modellen te trainen is het veel efficienter om hetzelfde model te gebruiken maar met meerdere outputs. 
In het voorbeeld hieronder kan je zien hoe je dit moet doen.

In [ ]:
import torch
import pandas as pd

# Load CSV data into a pandas DataFrame
data = pd.read_csv('data.csv')

# Convert DataFrame to NumPy arrays
features = data[['age', 'weight', 'height']].to_numpy()
targets = data[['score1', 'score2']].to_numpy()

# Convert NumPy arrays to PyTorch tensors
features = torch.from_numpy(features).float()
targets = torch.from_numpy(targets).float()

### Pytorch

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class MultiOutputRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3, 16)  # 3 input features, 16 hidden units
        self.fc2 = nn.Linear(16, 32)  # 16 hidden units, 32 hidden units
        self.fc3 = nn.Linear(32, 2)  # 32 hidden units, 2 output units (regression scores)  --> 2 outputs (regression)

    def forward(self, x):
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        return self.fc3(x)

loss_fn = F.mse_loss

model = MultiOutputRegressor()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model for a specified number of epochs
for epoch in range(100):
    # Forward pass
    outputs = model(features)

    # Calculate the loss
    loss = loss_fn(outputs, targets)

    # Zero the gradients
    optimizer.zero_grad()

    # Backward pass and update weights
    loss.backward()
    optimizer.step()

    # Print training progress
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}: Loss = {loss.item():.4f}')

### Tensorflow

In [ ]:
from tensorflow.keras import layers, Model

features = tf.convert_to_tensor(features, dtype=tf.float32)
targets = tf.convert_to_tensor(targets, dtype=tf.float32)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=2, input_shape=[3])
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(features, targets, epochs=10)


**Verschillende loss functies**

Voorgaande voorbeelden gebruikten dezelfde loss-functie voor de verschillende outputs. Dit omdat de voorspellingen identiek waren (beide outputs regressie).
Het is ook mogelijk om verschillende loss-functies te gebruiken voor elke output of voor verschillende subsets van outputs.

Onderstaande voorbeeld toont hoe dit eruit kan zien door middel van een regressieprobleem en een classificatie probleem met drie klassen te combineren.

In [ ]:
import torch
import pandas as pd

# Load CSV data into a pandas DataFrame
data = pd.read_csv('data2.csv')

# Convert DataFrame to NumPy arrays
features = data[['age', 'weight', 'height']].to_numpy()
targets_regression = data[['score']].to_numpy()
targets_cls = data['class'].astype('category').cat.codes.to_numpy() # zet a,b,c van dataset om naar 0,1,2

# Convert NumPy arrays to PyTorch tensors
features = torch.from_numpy(features).float()
targets_regression = torch.from_numpy(targets_regression).float()
targets_cls = torch.from_numpy(targets_cls).long()

In [ ]:
# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F

class CombinedModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(3, 16)  # 3 input features, 16 hidden units
        self.fc2 = nn.Linear(16, 32)  # 16 hidden units, 32 hidden units
        self.fc3_reg = nn.Linear(32, 1)  # 32 hidden units, 1 regression outputs
        self.fc3_cls = nn.Linear(32, num_classes)  # 32 hidden units, num_classes classification outputs

    def forward(self, x):
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        reg_output = self.fc3_reg(x)
        cls_output = self.fc3_cls(x)
        return reg_output, cls_output

def combined_loss(reg_pred, cls_pred, reg_target, cls_target):
    reg_loss = F.mse_loss(reg_pred, reg_target)
    cls_loss = F.cross_entropy(cls_pred, cls_target)
    return reg_loss + cls_loss

model = CombinedModel(num_classes=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    # Forward pass
    reg_pred, cls_pred = model(features)

    # Calculate the loss
    loss = combined_loss(reg_pred, cls_pred, targets_regression, targets_cls)

    # Zero the gradients
    optimizer.zero_grad()

    # Backward pass and update weights
    loss.backward()
    optimizer.step()

    # Print training progress
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}: Loss = {loss.item():.4f}')


In [ ]:
# Tensorflow
num_classes=3
inputs = Input(shape=(3,))
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)

# Regression output
reg_output = layers.Dense(1)(x)

# Classification output
cls_output = layers.Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=[reg_output, cls_output])

# Custom loss function combining MSE for regression and categorical crossentropy for classification
def custom_loss(y_true, y_pred):
    reg_loss = tf.keras.losses.mse(y_true[0], y_pred[0])
    cls_loss = tf.keras.losses.categorical_crossentropy(y_true[1], y_pred[1])
    return reg_loss + cls_loss

model.compile(loss=custom_loss, optimizer='adam')

# Convert y_cls to one-hot encoding
y_cls_one_hot = tf.one_hot(targets_cls, num_classes)

# Combine regression and classification targets
y = [targets_regression, y_cls_one_hot]

model.fit(features, y, epochs=10, batch_size=32)
